#Generacion de resumenes usando el modelo T5, Transformers, Seq2Seq y Fine Tuning

**Estudiante**: Rodrigo Walter Andre Basilio


### Introduccion

En este cuadernillo se usara un modelo de transformers bajo la arquitectura Seq2Seq (Modelo T5) para la **Generacion de Resumenes**.\
En esta tarea, se genera un resumen de un artículo o documento al pasarlo por una red. Hay 2 tipos de mecanismos de generación de resúmenes:

1. ***Resumen Extractivo:*** la red calcula las oraciones más importantes del artículo y las une para proporcionar la información más relevante del artículo.
2. ***Resumen Abstractivo***: la red crea nuevas oraciones para encapsular la esencia máxima del artículo y genera eso como salida. Las oraciones en el resumen pueden o no estar contenidas en el artículo.

En este cuadernillo se realizara la generacion de ***Resumenes Abstractivos***.

The notebook will be divided into separate sections to provide a organized walk through for the process used. This process can be modified for individual use cases. The sections are:

1. [Preparacion del entorno y importacion de Librerias](#section01)
2. [Preparando el Dataset para el procesamiento de datos: Clases](#section02)
3. [Aplicando Fine Tuning al Modelo: Función](#section03)
4. [Validando el rendimiento del modelo: Función](#section04)
5. [Main Function](#section05)
    * [Importing and Pre-Processing the domain data](#section502)
    * [Creation of Dataset and Dataloader](#section503)
    * [Neural Network and Optimizer](#section504)
    * [Training Model and Logging to WandB](#section505)
    * [Validation and generation of Summary](#section506)
6. [Examples of the Summary Generated from the model](#section06)


#### Technical Details

This script leverages on multiple tools designed by other teams. Details of the tools used below. Please ensure that these elements are present in your setup to successfully implement this script.

- **Data**:
	- Estamos utilizando el conjunto de datos de resúmenes de noticias disponible en [Kaggle](https://www.kaggle.com/datasets/gabrieltardochi/wikipedia-movie-plots-with-plot-summaries)
	- Este dataset es una colección creada a partir de descripciones de tramas de peliculas, extrayendo lassiguientes columnas.
	- este dataset cuenta con `34000` filas de datos. Cada fila contiene los siguientes datos:
		- **Realease Year** : Año de estreno de la Pelicula
		- **Title** : titulo de la Pelicula
		- **Origin/Ethnicity**: Pais o Etnia de origen de la Pelicula
		- **Director** : Director de la Pelicula
		- **Cast**: Reparto de Actores de la Pelicula
		- **Genre**: Genero Cinematografico de la Pelicula
		- **Wiki Page**: Enlace a la pagina de wikipedia de la Pelicula
		- **Plot**:  Descripcion de la trama de la Pelicula
		- **Plot Summary**: Resumen original de la Pelicula

- **Modelo de Lenguaje Utilizado**:
    - Este cuadernillo utiliza uno de los modelos de transformers más recientes y novedosos ***T5***. [Paper](https://arxiv.org/abs/1910.10683)    
    - ***T5***: es, en muchos aspectos, una arquitectura de transformadores única que no solo ofrece resultados de vanguardia en muchas tareas de procesamiento del lenguaje natural (NLP, por sus siglas en inglés), sino que también tiene un enfoque muy radical para abordar estas tareas.\
	-**Seq2Seq (Sequence-to-Sequence)**: es un tipo de modelo de aprendizaje automático que se utiliza para problemas de traducción, generación de resúmenes, chatbots y otras tareas de procesamiento del lenguaje natural. Consiste en dos componentes principales: un codificador que toma una secuencia de entrada y la transforma en un vector de contexto, y un decodificador que genera una secuencia de salida basada en ese vector de contexto.\
	En el caso del **T5**, ha sido pre-entrenado en una amplia variedad de tareas de procesamiento de texto utilizando una formulación de texto a texto. Esto significa que puede ser utilizado tanto para tareas de codificación como de decodificación, lo que lo convierte en un modelo **Seq2Seq** completo.
    - **Text-2-Text**: Según el gráfico extraído del artículo de T5, todas las tareas de NLP se convierten en un problema de text-to-text. Tareas como la traducción, clasificación, resumen y respuesta a preguntas, todas se tratan como problemas de conversión de texto a texto en lugar de considerarse declaraciones de problemas únicos y separados.
	- **Enfoque unificado para el aprendizaje profundo de NLP**: Dado que la tarea se refleja puramente en la entrada y salida de texto, se puede utilizar el mismo modelo, objetivo, procedimiento de entrenamiento y proceso de decodificación para CUALQUIER tarea. El marco mencionado anteriormente se puede utilizar para cualquier tarea, como preguntas y respuestas, resumen, etc.\
		Se tomara la información del artículo de T5 para preparar el dataset antes de realizar el Fine tuning y el entrenamiento.
     
   - [Documentacion del Modelo T5 para Python](https://huggingface.co/transformers/model_doc/t5.html)

![**Each NLP problem as a “text-to-text” problem** - input: text, output: text](https://miro.medium.com/max/4006/1*D0J1gNQf8vrrUpKeyD8wPA.png)


<a id='section01'></a>
### Preparacion del entorno y importacion de Librerias
Primero conectaremos nuestro drive para poder utilizar el dataset desde este almacenamiento

Luego instalaremos las bibliotecas necesarias y luego importaremos las bibliotecas y módulos necesarios para ejecutar nuestro script. Instalaremos:
* Transformers
* SentencePiece

Las librerias a importar son:
* Pandas
* Pytorch
* Pytorch Utils para el Dataset y el Dataloader
* Transformers
* T5 Model y Tokenizer

A Continuación, prepararemos el dispositivo para la ejecución de CUDA. Esta configuración es necesaria si se desea aprovechar la GPU incorporada. Primero verificaremos la GPU disponible utilizando el comando "nvidia" y luego definiremos nuestro dispositivo.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers -q
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.6 MB/s eta 0:00:00


In [3]:
# Importando librerias
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importando los modulos T5 desde huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [4]:
# Comprobando si tenemos acceso a una GPU
!nvidia-smi

Fri Jun 23 09:58:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# # Configurando el dispositivo para utilizar la GPU.
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'


<a id='section02'></a>
### Preparando el Dataset para el procesamiento de datos: Clases

Comenzaremos con la creación de la clase Dataset, que define cómo se preprocesa el texto antes de enviarlo a la red neuronal. Este conjunto de datos será utilizado por el método Dataloader, que alimentará los datos en lotes a la red neuronal para su entrenamiento y procesamiento adecuados.
El Dataloader y el Dataset seran usados dentro del `main()`.
El Dataloader y el Dataset son constructores de la biblioteca PyTorch para definir y controlar el preprocesamiento de datos y su paso a la red neuronal.  

#### *Dataset Custom* Clase Dataset
- Esta clase se define para aceptar el DataFrame como entrada y generar una salida tokenizada que es utilizada por el modelo **T5** para el entrenamiento.
- Se esta utilizando el tokenizador **T5** para tokenizar los datos en las columnas `Plot` y `Plot Summary` del DataFrame. El tokenizador utiliza el método `batch_encode_plus` para realizar la tokenización y generar las salidas necesarias, a saber: `source_id`, `source_mask` a partir del texto real y `target_id` y `target_mask` a partir del resumen de texto original.
- La Clase *CustomDataset* se utiliza para crear 2 conjuntos de datos, uno para entrenamiento y otro para validación.
- El *Training Dataset* se utiliza para el fine tune del modelo: **el 80% de los datos originales**
- El *Validation Dataset* se utiliza para evaluar el rendimiento del modelo. El modelo no ha visto estos datos durante el entrenamiento.

#### Dataloader: Usado dentro del `main()`
- El Dataloader se utiliza para crear dataloaders de entrenamiento y validación que cargan datos en la red neuronal de manera definida. Esto es necesario porque no se puede cargar todos los datos del conjunto de datos en la memoria de una vez, por lo tanto, la cantidad de datos cargados en la memoria y luego pasados a la red neuronal debe ser controlada.
- Este control se logra utilizando parámetros como el tamaño del lote `batch_size` y la longitud máxima `max_len`.
- Los dataloaders de entrenamiento y validación se utilizan en la parte de entrenamiento y validación del flujo, respectivamente.

In [6]:
# Creando un conjunto de datos personalizado para leer el DataFrame y cargarlo en el dataloader para pasarlo a la red neuronal en una etapa posterior,
# con el fin de ajustar el modelo y prepararlo para realizar predicciones.

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.PlotSummary
        self.ctext = self.data.Plot

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long),
            'source_mask': source_mask.to(dtype=torch.long),
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

<a id='section03'></a>
### Aplicando Fine Tuning al Modelo: Función

Aquí definimos una función de entrenamiento que entrena el modelo en el conjunto de datos de entrenamiento creado anteriormente, durante un número especificado de veces (EPOCH). Un epoch define cuántas veces se pasará el conjunto de datos completo a través de la red.

Esta función se llama en la función principal  `main()`

los siguientes eventos ocurren para aplicar fine tune a la red neuronal:
- El epoch, el tokenizador, el modelo, los detalles del dispositivo, el dataloader de prueba y el optimizador se pasan a la función `train()` cuando se llama desde la función principal `main()`.
- El dataloader pasa los datos al modelo en función del tamaño del lote (batch size).
- Se calculan las etiquetas del modelo de lenguaje `language_model_labels` apartir del `target_ids`, también se extraen `source_id` y `attention_mask`.
- El primer elemento de las salidas del modelo devuelve la pérdida (loss) para el paso hacia adelante (forward pass).
- El valor de pérdida se utiliza para optimizar los pesos de las neuronas en la red.
- despues de cada 10 pasos (Steps) , se registra el valor de pérdida en el servicio wandb. Este registro luego se utiliza para generar gráficos para su análisis.
- Después de cada 500 pasos (Steps), se imprime el valor de pérdida en la consola.

In [7]:
# Creando la función de entrenamiento. Esta se llamará en la función principal. Se ejecuta en función del valor del epoch.
# El modelo se pone en modo de entrenamiento y luego se itera sobre el entrenamiento del dataloader y se pasa a la red definida.

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


<a id='section04'></a>
### Validando el rendimiento del modelo: Función

Durante la etapa de validación, pasamos los datos no vistos (Testing Dataset), el modelo entrenado, el tokenizador y los detalles del dispositivo a la función para realizar la ejecución de validación. Este paso genera un nuevo resumen para el conjunto de datos que no ha sido visto durante la sesión de entrenamiento.

Esta función se llama en la función principal `main()`

Estos datos no vistos son el 20% que se separó durante la creación del Dataset. Durante la etapa de validación, no se actualizan los pesos del modelo. Utilizamos el método generate para generar nuevo texto para el resumen.

Esto depende del método de codificación `Beam-Search coding` desarrollado para la generación de secuencias para modelos con encabezado de LM.

El texto generado y el resumen original se decodifican de tokens a texto y se devuelven a la función principal `main()`

In [8]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask,
                max_length=150,
                num_beams=2,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

<a id='section05'></a>
### Funcion Principal: Main()

La función `main()` es como su nombre indica, el centro donde se ejecutan todas las funciones/flujos creados anteriormente en el cuadernillo. Los siguientes pasos se ejecutan en la función `main()`:


<a id='section502'></a>
#### Importación y preprocesamiento de los datos del dominio:

* Importamos el archivo y lo convertimos en un dataframe, asignándole los encabezados según la documentación.
* Limpiamos el archivo para eliminar las columnas no deseadas.
* Agregamos una nueva cadena al inicio de la columna principal del artículo:  `summarize: ` Esto se hace porque **T5** tiene un formato similar al el summarization dataset.
* Imprimimos las 5 primeras filas del dataframe en la consola.

<a id='section503'></a>
#### Creación de Dataset y Dataloader:

* El dataframe actualizado se divide en una proporción de 80-20 para pruebas y validación.
* Ambos dataframes se pasan a la clase `CustomerDataset` para la tokenización de los nuevos artículos y sus resúmenes.
* La tokenización se realiza utilizando los parámetros de longitud pasados a la clase.
* Se definen y pasan los parámetros de entrenamiento y validación al constructor Dataloader de `pytorch Dataloader contstruct`para crear los dataloaders de `train` y `validation`.
* Estos dataloaders se pasarán a `train()` y `validate()` respectivamente para la acción de entrenamiento y validación.
* Se imprime la forma de los datasets en la consola.


<a id='section504'></a>
#### Red neuronal y optimizador:

* En esta etapa, definimos el modelo y el optimizador que se utilizarán para el entrenamiento y para actualizar los pesos de la red.
* Estamos utilizando el modelo de transformers `t5-base`.
* Utilizamos el comando `T5ForConditionalGeneration.from_pretrained("t5-base")` para definir nuestro modelo.\
`T5ForConditionalGeneration`  agrega una cabeza de modelo de lenguaje a nuestro modelo`T5 model`. La cabeza de modelo de lenguaje nos permite generar texto basado en el entrenamiento del modelo `T5 model`.
* Se utiliza el optimizador `Adam`.

<a id='section505'></a>
#### Entrenamiento del modelo:

* Llamamos a la función `train()` con todos los parámetros necesarios.
* Se imprime en la consola la pérdida cada 500 pasos.


<a id='section506'></a>
#### Validación y generación de resumen:

* Después de completar el entrenamiento, se inicia la etapa de validación.
* Como se define en la función de validación, los pesos del modelo no se actualizan. Utilizamos el modelo con el fine tuning aplicado para generar nuevos resúmenes basados en el texto del artículo.
* Se imprime en la consola una salida que indica cuántos pasos se han completado después de cada paso 100.
* El resumen original y el resumen generado se convierten en una lista y se devuelven a la función main.
* Ambas listas se utilizan para crear el dataframe final con 2 columnas: **Resumen Generado** y **Resumen Original**
* El dataframe se guarda como un archivo CSV en el Drive.

In [9]:
def main():

    TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
    VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
    TRAIN_EPOCHS = 2        # number of epochs to train (default: 10)
    VAL_EPOCHS = 1
    LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
    SEED = 42               # random seed (default: 42)
    MAX_LEN = 512
    SUMMARY_LEN = 150

    # Establecer semillas aleatorias y determinismo en PyTorch para reproducibilidad
    torch.manual_seed(SEED) # semilla aleatoria pytorch
    np.random.seed(SEED) # semilla aleatoria numpy
    torch.backends.cudnn.deterministic = True

    # tokenizador para codificar el texto
    tokenizer = T5Tokenizer.from_pretrained("t5-base")


    # Importando y preprocesando los datos del dominio
    # Seleccionando solo las columnas necesarias.
    # Agregando el texto de resumen al frente del texto. Esto es para formatear el conjunto de datos de manera similar a cómo se entrenó el modelo T5 para la tarea de resumen.
    df = pd.read_csv('/content/drive/MyDrive/final_t5/wiki_movie_plots_deduped_with_summaries.csv',encoding='latin-1')
    df = df[['Plot','PlotSummary']]
    df.ctext = 'summarize: ' + df.Plot
    print(df.head())

    df = df.head(10)  # se define cuantas filas se usaran

    # Se crea el Dataset y el Dataloader
    # Definiendo el tamaño de entrenamiento, el 80% de los datos se utilizarán para entrenamiento y el resto se utilizará para validación.
    train_size = int(len(df) * 0.8)  # Tamaño del conjunto de entrenamiento (80%)
    train_dataset = df[:train_size].reset_index(drop=True)  # Primeros 80% de los datos
    val_dataset = df[train_size:].reset_index(drop=True)  # Resto de los datos para validación


    print("FULL Dataset: {}".format(df.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(val_dataset.shape))


    # Creando los conjuntos de datos de entrenamiento y validación para su posterior creación del Dataloader.
    training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)

    # Definiendo los parámetros para la creación de los dataloaders.
    train_params = {
        'batch_size': TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

    val_params = {
        'batch_size': VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

    # Creación de los dataloaders para pruebas y validación. Esto se utilizará posteriormente en la etapa de entrenamiento y validación del modelo.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)



    # Definiendo el modelo. Estamos utilizando el modelo t5-base y se agregó una capa de modelo de lenguaje encima para la generación del resumen.
    # Además, este modelo se envía al dispositivo (GPU/TPU) para utilizar el hardware.
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    model = model.to(device)

    # Definiendo el optimizador que se utilizará para ajustar los pesos de la red durante la sesión de entrenamiento.
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

    # Bucle de entrenamiento
    print('Initiating Fine-Tuning for the model on our dataset')

    for epoch in range(TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)


    # Bucle de validación y guardado del archivo resultante con las predicciones y los valores reales en un dataframe.
    # Se guarda el Dataframe como predictions.csv
    print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
    for epoch in range(VAL_EPOCHS):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Summmary':predictions,'Original Summary':actuals})
        final_df.to_csv('predictions.csv')
        print('Output Files generated for review')

if __name__ == '__main__':
    main()

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
<ipython-input-9-ac734bb28a33>:26: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.ctext = 'summarize: ' + df.Plot


                                                Plot  \
0  A bartender is working at a saloon, serving dr...   
1  The moon, painted with a smiling face hangs ov...   
2  The film, just over a minute long, is composed...   
3  Lasting just 61 seconds and consisting of two ...   
4  The earliest known adaptation of the classic f...   

                                         PlotSummary  
0  Carrie Nation and her followers burst into a s...  
1  The moon, painted with a smiling face hangs ov...  
2  The film, just over a minute long, is composed...  
3  The first shot is set in a wood during winter ...  
4  The earliest known adaptation of the classic f...  
FULL Dataset: (10, 2)
TRAIN Dataset: (8, 2)
TEST Dataset: (2, 2)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Initiating Fine-Tuning for the model on our dataset


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  10.860650062561035
Epoch: 1, Loss:  2.3727567195892334
Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe
Completed 0
Output Files generated for review


<a id='section06'></a>
### Ejemplos de resumenes generados por este Modelo

##### Ejemplo #1

**Texto Original**\
The opening scene shows the interior of the robbers' den. The walls are decorated with the portraits of notorious criminals and pictures illustrating the exploits of famous bandits. Some of the gang are lounging about, while others are reading novels and illustrated papers. Although of youthful appearance, each is dressed like a typical Western desperado. The "Bandit Queen" leading a blindfolded new recruit, now enters the room. He is led to the center of the room, raises his right hand and is solemnly sworn in. When the bandage is removed from his eyes he finds himself looking into the muzzles of a dozen or more 45's. The gang then congratulates the new member and heartily shake his hand. The "Bandit Queen" who is evidently the leader of the gang, now calls for volunteers to hold up a train. All respond, but she picks out seven for the job who immediately leave the cabin.
The next scene shows the gang breaking into a barn. They steal ponies and ride away. Upon reaching the place agreed upon they picket their ponies and leaving them in charge of a trusted member proceed to a wild mountain spot in a bend of the railroad, where the road runs over a steep embankment. The spot is an ideal one for holding up a train. Cross ties are now placed on the railroad track and the gang hide in some bushes close by and wait for the train. The train soon approaches and is brought to a stop. The engineer leaves his engine and proceeds to remove the obstruction on the track. While he is bending over one of the gang sneaks up behind them and hits him on the head with an axe, and knocks him senseless down the embankment, while the gang surround the train and hold up the passengers. After securing all the "valuables," consisting principally of candy and dolls, the robbers uncouple the engine and one car and make their escape just in time to avoid a posse of police who appear on the scene. Further up the road they abandon the engine and car, take to the woods and soon reach their ponies.
In the meantime the police have learned the particulars of the hold-up from the frightened passengers and have started up the railroad tracks after the fleeing robbers. The robbers are next seen riding up the bed of a shallow stream and finally reach their den, where the remainder of the gang have been waiting for them. Believing they have successfully eluded their pursuers, they proceed to divide the "plunder." The police, however, have struck the right trail and are in close pursuit. While the "plunder" is being divided a sentry gives the alarm and the entire gang, abandoning everything, rush from the cabin barely in time to escape capture. The police make a hurried search and again start in pursuit. The robbers are so hard pressed that they are unable to reach their ponies, and are obliged to take chances on foot. The police now get in sight of the fleeing robbers and a lively chase follows through tall weeds, over a bridge and up a steep hill. Reaching a pond the police are close on their heels. The foremost robbers jump in clothes and all and strike out for the opposite bank. Two hesitate and are captured. Boats are secured and after an exciting tussle the entire gang is rounded up. In the mix up one of the police is dragged overboard. The final scene shows the entire gang of bedraggled and crestfallen robbers tied together with a rope and being led away by the police. Two of the police are loaded down with revolvers, knives and cartridge belts, and resemble walking aresenals. As a fitting climax a confederate steals out of the woods, cuts the rope and gallantly rescues the "Bandit Queen."


**Resumen Original**\
The opening scene shows the interior of the robbers' den. The "Bandit Queen" who is evidently the leader of the gang, now calls for volunteers to hold up a train. After securing all the "valuables," consisting principally of candy and dolls, the robbers make their escape just in time to avoid a posse of police who appear on the scene.

**Resumen Generado**\
the gang is now in charge of a new member. The engineer leaves his engine and removes the obstruction on the track. While he is bending over one of the gang sneaks up behind him and knocks him senseless down the embankment, while the gang surround the train and hold up the passengers. After securing all the "values," consisting principally of candy and dolls, the robbers abandon the engine and one car and make their escape just in time to avoid

##### Ejemplo #2

**Texto Original**\
Scenes are introduced using lines of the poem.[2] Santa Claus, played by Harry Eytinge, is shown feeding real reindeer[4] and finishes his work in the workshop. Meanwhile, the children of a city household hang their stockings and go to bed, but unable to sleep they engage in a pillow fight. Santa Claus leaves his home on a sleigh with his reindeer. He enters the children's house through the chimney, and leaves the presents. The children come down the stairs and enjoy their presents


**Resumen Original**\
Scenes are introduced using lines of the poem. Santa Claus, played by Harry Eytinge, is shown feeding real reindeer and finishes his work in the workshop. Meanwhile, the children of a city household hang their stockings and go to bed, but unable to sleep they engage in a pillow fight.

**Resumen Generado**\
real reindeer. He finishes his work in the workshop. The children hang their stockings and go to bed, but unable to sleep they engage in a pillow fight. Santa Claus leaves his reindeer with his reindeer. He enters the children's house through the chimney, and leaves the presents. The children come down the stairs and enjoy their presents.